# Customer Segmentation with K-Means Clustering

Customer segmentation will be applied to an e-commerce customer database using K-means clustering from scikit-learn. It is an [extension](https://github.com/cereniyim/Data-Science-Projects/blob/master/ECommerce-Sales-Data-EDA/ECommerce_Sales_Data_Analysis.ipynb) of a case study solved couple of months ago. 

The provided customers database is visualized as part of a case study. This project is taking the case study one step further with the following motive:

**Can this customer database be grouped to develop customized relationships?** 

**To answer this question 3 features will be created and used:** <br>
- products ordered
- average return rate
- total spending

**Dataset represents real customers & orders data between November 2018 - April 2019 and it is pseudonymized for confidentiality.**

**Imports**

In [14]:
# data wrangling
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# for data preprocessing and clustering
from sklearn.cluster import KMeans

%matplotlib inline
# to include graphs inline within the frontends next to code

%config InlineBackend.figure_format='retina'
#to enable retina (high resolution) plots

pd.options.mode.chained_assignment = None
# to bypass warnings in various dataframe assignments

**Investigate data**

In [15]:
# load data into a dataframe
customers_orders = pd.read_csv("Orders - Analysis Task.csv")

In [16]:
# first rows of the dataset
customers_orders.head()

,product_title,product_type,variant_title,variant_sku,variant_id,customer_id,order_id,day,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
0,DPR,DPR,100,AD-982-708-895-F-6C894FB,52039657.0,1312378.0,8.329072e+13,04/12/2018,2.0,200.0,-200.00,0.00,0.0,0.0,0.0,0.0,2.0
1,RJF,Product P,28 / A / MTM,83-490-E49-8C8-8-3B100BC,56914686.0,3715657.0,3.625379e+13,01/04/2019,2.0,190.0,-190.00,0.00,0.0,0.0,0.0,0.0,2.0
2,CLH,Product B,32 / B / FtO,68-ECA-BC7-3B2-A-E73DE1B,24064862.0,9533448.0,7.309456e+13,05/11/2018,0.0,164.8,-156.56,-8.24,0.0,0.0,0.0,-2.0,2.0
3,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868.0,4121004.0,5.361658e+13,19/02/2019,1.0,119.0,-119.00,0.00,0.0,0.0,0.0,0.0,1.0
4,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868.0,4121004.0,2.926322e+13,19/02/2019,1.0,119.0,-119.00,0.00,0.0,0.0,0.0,0.0,1.0


In [17]:
# first glance of customers_orders data
customers_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15586 entries, 0 to 15585
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   product_title           15586 non-null  object 
 1   product_type            15586 non-null  object 
 2   variant_title           15585 non-null  object 
 3   variant_sku             15585 non-null  object 
 4   variant_id              15585 non-null  float64
 5   customer_id             15585 non-null  float64
 6   order_id                15585 non-null  float64
 7   day                     15585 non-null  object 
 8   net_quantity            15585 non-null  float64
 9   gross_sales             15585 non-null  float64
 10  discounts               15585 non-null  float64
 11  returns                 15585 non-null  float64
 12  net_sales               15585 non-null  float64
 13  taxes                   15585 non-null  float64
 14  total_sales             15585 non-null

In [18]:
# descriptive statistics of the non-object columns
customers_orders.describe()

,variant_id,customer_id,order_id,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
count,1.558500e+04,1.558500e+04,1.558500e+04,15585.000000,15585.000000,15585.000000,15585.000000,15585.000000,15585.000000,15585.000000,15585.000000,15585.000000
mean,2.548774e+11,5.751895e+11,5.492515e+13,1.013282,81.012848,-17.746737,-0.046019,63.220092,12.538026,75.758118,-0.001155,1.014437
std,4.286562e+12,6.182477e+12,2.581021e+13,0.128730,13.326769,5.566007,1.881968,13.272718,2.823165,15.822421,0.037555,0.125065
min,1.001447e+07,1.001914e+06,1.000657e+13,0.000000,29.160000,-200.000000,-142.500000,0.000000,0.000000,0.000000,-2.000000,1.000000
25%,2.582628e+07,3.306370e+06,3.256120e+13,1.000000,74.170000,-20.840000,0.000000,58.330000,11.670000,70.000000,0.000000,1.000000
50%,4.483018e+07,5.534637e+06,5.511440e+13,1.000000,79.170000,-16.500000,0.000000,60.730000,11.870000,71.250000,0.000000,1.000000
75%,7.443784e+07,7.815352e+06,7.687706e+13,1.000000,81.660000,-14.840000,0.000000,64.000000,12.670000,76.000000,0.000000,1.000000
max,8.422212e+13,9.954992e+13,9.999554e+13,4.000000,316.680000,-10.750000,0.000000,295.830000,59.170000,355.000000,0.000000,4.000000


There were significant number of rows whose `ordered_item_quantity` is 0 and `net_quantity` is less than 0, which means they are not ordered/sold at all; but the fact that they have returns requires investigation.

In [63]:
print("Number of rows that net quantity is negative:", 
      customers_orders[customers_orders.net_quantity < 0].shape[0])

Number of rows that net quantity is negative: 0


In [20]:
a = np.array([[1,1,1],[2,2,2],[3,3,3]])
a.shape

(3, 3)

**These rows will be excluded from the orders dataset for the project.**

In [21]:
# exclude not sold/ordered SKUs from the dataset
customers_orders = customers_orders[customers_orders["ordered_item_quantity"] > 0]

## 1. Products ordered
It is the count of the products ordered in product_type column by a customer. <br>

**Create functions to identify customers who order multiple products**

In [22]:
def encode_column(column):
    if column > 0:
        return 1
    if column <= 0:
        return 0


def aggregate_by_ordered_quantity(dataframe, column_list):
    '''this function:
    1. aggregates a given dataframe by column list, 
    as a result creates a aggregated dataframe by counting the ordered item quantities

    2. adds number_of_X ordered where X is the second element in the column_list 
    to the aggregated dataframe by encoding ordered items into 1

    3. creates final dataframe containing information about 
    how many of X are ordered, based on the first element passed in the column list'''

    aggregated_dataframe = (dataframe
                            .groupby(column_list)
                            .ordered_item_quantity.count()
                            .reset_index())

    aggregated_dataframe["products_ordered"] = (aggregated_dataframe
                                                 .ordered_item_quantity
                                                 .apply(encode_column))

    final_dataframe = (aggregated_dataframe
                       .groupby(column_list[0])
                       .products_ordered.sum() # aligned with the added column name
                       .reset_index())

    return final_dataframe

In [23]:
# apply functions to customers_orders
customers = aggregate_by_ordered_quantity(customers_orders, ["customer_id", "product_type"])

In [24]:
print(customers.head())

   customer_id  products_ordered
0    1001914.0                 1
1    1002167.0                 1
2    1003728.0                 2
3    1003899.0                 3
4    1006436.0                 1


## 2. Average Return Rate
It is the ratio of returned item quantity and ordered item quantity. This ratio is first calculated per order and then averaged for all orders of a customer.

In [64]:
from pandas._libs.hashtable import objects_are_equal
# aggregate data per customer_id and order_id, 
# to see ordered item sum and returned item sum
ordered_sum_by_customer_order = (customers_orders
                                 .groupby(["customer_id", "order_id"])
                                 .ordered_item_quantity.sum()
                                 .reset_index())

returned_sum_by_customer_order = (customers_orders
                                  .groupby(["customer_id", "order_id"])
                                  .returned_item_quantity.sum()
                                  .reset_index())

# merge two dataframes to be able to calculate unit return rate
ordered_returned_sums = pd.merge(ordered_sum_by_customer_order, returned_sum_by_customer_order)

<bound method NDFrame.head of         customer_id      order_id  ordered_item_quantity  \
0      1.001914e+06  7.975857e+13                    1.0   
1      1.002167e+06  5.744015e+13                    1.0   
2      1.003728e+06  3.446249e+13                    1.0   
3      1.003728e+06  8.730239e+13                    1.0   
4      1.003899e+06  2.696881e+13                    2.0   
...             ...           ...                    ...   
12908  9.636748e+13  3.550268e+13                    1.0   
12909  9.649252e+13  7.042030e+13                    1.0   
12910  9.719951e+13  2.809104e+13                    1.0   
12911  9.736067e+13  6.514700e+13                    1.0   
12912  9.954992e+13  5.524459e+13                    1.0   

       returned_item_quantity  
0                         0.0  
1                         0.0  
2                         0.0  
3                         0.0  
4                         0.0  
...                       ...  
12908                    

In [26]:
# calculate unit return rate per order and customer
ordered_returned_sums["average_return_rate"] = (-1 * 
                                             ordered_returned_sums["returned_item_quantity"] /
                                             ordered_returned_sums["ordered_item_quantity"])

In [27]:
ordered_returned_sums.head()

,customer_id,order_id,ordered_item_quantity,returned_item_quantity,average_return_rate
0,1001914.0,7.975857e+13,1.0,0.0,-0.0
1,1002167.0,5.744015e+13,1.0,0.0,-0.0
2,1003728.0,3.446249e+13,1.0,0.0,-0.0
3,1003728.0,8.730239e+13,1.0,0.0,-0.0
4,1003899.0,2.696881e+13,2.0,0.0,-0.0


In [28]:
# take average of the unit return rate for all orders of a customer
customer_return_rate = (ordered_returned_sums
                        .groupby("customer_id")
                        .average_return_rate
                        .mean()
                        .reset_index())

In [29]:
return_rates = pd.DataFrame(customer_return_rate["average_return_rate"]
                            .value_counts()
                            .reset_index())

return_rates.rename(columns=
                    {"index": "average return rate",
                     "average_return_rate": "count of unit return rate"},
                    inplace=True)

return_rates.sort_values(by="average return rate")

,average return rate,count of unit return rate
0,0.000000,10039
2,0.333333,2
3,0.500000,1
4,0.666667,1
1,1.000000,8


In [30]:
# add average_return_rate to customers dataframe
customers = pd.merge(customers,
                     customer_return_rate,
                     on="customer_id")

## 3. Total spending
Total spending is the aggregated sum of total sales value which is the amount after the taxes and returns.

In [31]:
# aggreagate total sales per customer id
customer_total_spending = (customers_orders
                           .groupby("customer_id")
                           .total_sales
                           .sum()
                           .reset_index())

customer_total_spending.rename(columns = {"total_sales" : "total_spending"},
                               inplace = True)

## Create features data frame

In [32]:
# add total sales to customers dataframe
customers = customers.merge(customer_total_spending, 
                            on="customer_id")

In [33]:
print("The number of customers from the existing customer base:", customers.shape[0])

The number of customers from the existing customer base: 10051


In [34]:
# drop id column since it is not a feature
customers.drop(columns="customer_id",
               inplace=True)

In [35]:
customers.head()

,products_ordered,average_return_rate,total_spending
0,1,0.0,79.2
1,1,0.0,79.2
2,2,0.0,194.2
3,3,0.0,386.8
4,1,0.0,70.0


### Visualize features

In [36]:
fig = make_subplots(rows=3, cols=1,
                   subplot_titles=("Products Ordered", 
                                   "Average Return Rate", 
                                   "Total Spending"))

fig.append_trace(go.Histogram(x=customers.products_ordered),
                 row=1, col=1)

fig.append_trace(go.Histogram(x=customers.average_return_rate),
                 row=2, col=1)

fig.append_trace(go.Histogram(x=customers.total_spending),
                 row=3, col=1)

fig.update_layout(height=800, width=800,
                  title_text="Distribution of the Features")

fig.show()

## Scale Features: Log Transformation

In [37]:
def apply_log1p_transformation(dataframe, column):
    '''This function takes a dataframe and a column in the string format
    then applies numpy log1p transformation to the column
    as a result returns log1p applied pandas series'''
    
    dataframe["log_" + column] = np.log1p(dataframe[column])
    return dataframe["log_" + column]

### 1. Products ordered

In [38]:
apply_log1p_transformation(customers, "products_ordered")

0        0.693147
1        0.693147
2        1.098612
3        1.386294
4        0.693147
           ...   
10046    0.693147
10047    0.693147
10048    0.693147
10049    0.693147
10050    0.693147
Name: log_products_ordered, Length: 10051, dtype: float64

### 2. Average return rate

In [65]:
apply_log1p_transformation(customers, "average_return_rate")

0        0.000000e+00
1        0.000000e+00
2        0.000000e+00
3        0.000000e+00
4        0.000000e+00
             ...     
10050    0.000000e+00
10051    6.210218e-05
10052    7.372575e-18
10053    2.293201e-04
10054    3.364446e-01
Name: log_average_return_rate, Length: 10055, dtype: float64

### 3. Total spending

In [40]:
apply_log1p_transformation(customers, "total_spending")

0        4.384524
1        4.384524
2        5.274025
3        5.960490
4        4.262680
           ...   
10046    4.174387
10047    4.343805
10048    4.215824
10049    4.330733
10050    4.174387
Name: log_total_spending, Length: 10051, dtype: float64

### Visualize log transformation applied features

In [66]:
fig = make_subplots(rows=3, cols=1,
                   subplot_titles=("Products Ordered", 
                                   "Average Return Rate", 
                                   "Total Spending"))

fig.append_trace(go.Histogram(x=customers.log_products_ordered),
                 row=1, col=1)

fig.append_trace(go.Histogram(x=customers.log_average_return_rate),
                 row=2, col=1)

fig.append_trace(go.Histogram(x=customers.log_total_spending),
                 row=3, col=1)

fig.update_layout(height=800, width=800,
                  title_text="Distribution of the Features after Logarithm Transformation")

fig.show()

In [42]:
customers.head()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending
0,1,0.0,79.2,0.693147,0.0,4.384524
1,1,0.0,79.2,0.693147,0.0,4.384524
2,2,0.0,194.2,1.098612,0.0,5.274025
3,3,0.0,386.8,1.386294,0.0,5.960490
4,1,0.0,70.0,0.693147,0.0,4.262680


In [43]:
# features we are going to use as an input to the model
customers.iloc[:,3:]

,log_products_ordered,log_average_return_rate,log_total_spending
0,0.693147,0.0,4.384524
1,0.693147,0.0,4.384524
2,1.098612,0.0,5.274025
3,1.386294,0.0,5.960490
4,0.693147,0.0,4.262680
...,...,...,...
10046,0.693147,0.0,4.174387
10047,0.693147,0.0,4.343805
10048,0.693147,0.0,4.215824
10049,0.693147,0.0,4.330733


## Create K-means model

In [44]:
# create initial K-means model
kmeans_model = KMeans(init='k-means++', 
                      max_iter=500, 
                      random_state=42)

In [45]:
kmeans_model.fit(customers.iloc[:,3:])

# print the sum of distances from all examples to the center of the cluster
print("within-cluster sum-of-squares (inertia) of the model is:", kmeans_model.inertia_)

within-cluster sum-of-squares (inertia) of the model is: 162.21148002168394


## Hyperparameter tuning: Find optimal number of clusters

In [46]:
def make_list_of_K(K, dataframe):
    '''inputs: K as integer and dataframe
    apply k-means clustering to dataframe
    and make a list of inertia values against 1 to K (inclusive)
    return the inertia values list
    '''
    
    cluster_values = list(range(1, K+1))
    inertia_values=[]
    
    for c in cluster_values:
        model = KMeans(
            n_clusters = c, 
            init='k-means++', 
            max_iter=500, 
            random_state=42)
        model.fit(dataframe)
        inertia_values.append(model.inertia_)
    
    return inertia_values

### Visualize different K and models

In [67]:
# save inertia values in a dataframe for k values between 1 to 15 
results = make_list_of_K(15, customers.iloc[:, 3:])
print(results)

[18500.605169605322, 2852.8961892179204, 928.0594936339055, 450.2609120400752, 347.83908871556645, 260.9373824728642, 211.36885473699067, 185.4590602692158, 161.13069878138106, 135.24447416027408, 120.58517788142251, 100.5976399902913, 91.30099945160829, 80.98444441062716, 70.44675693310477]


In [47]:
# save inertia values in a dataframe for k values between 1 to 15 
results = make_list_of_K(15, customers.iloc[:, 3:])

k_values_distances = pd.DataFrame({"clusters": list(range(1, 16)),
                                   "within cluster sum of squared distances": results})

In [48]:
# visualization for the selection of number of segments
fig = go.Figure()

fig.add_trace(go.Scatter(x=k_values_distances["clusters"], 
                         y=k_values_distances["within cluster sum of squared distances"],
                         mode='lines+markers'))

fig.update_layout(xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1),
                  title_text="Within Cluster Sum of Squared Distances VS K Values",
                  xaxis_title="K values",
                  yaxis_title="Cluster sum of squared distances")

fig.show()

## Update K-Means Clustering

In [49]:
# create clustering model with optimal k=4
updated_kmeans_model = KMeans(n_clusters = 4, 
                              init='k-means++', 
                              max_iter=500, 
                              random_state=42)

updated_kmeans_model.fit_predict(customers.iloc[:,3:])

array([0, 0, 2, ..., 0, 0, 0], dtype=int32)

### Add cluster centers to the visualization

In [50]:
# create cluster centers and actual data arrays
cluster_centers = updated_kmeans_model.cluster_centers_
actual_data = np.expm1(cluster_centers)
add_points = np.append(actual_data, cluster_centers, axis=1)
add_points

array([[1.00037265e+00, 6.21041045e-05, 7.26876695e+01, 6.93333487e-01,
        6.21021762e-05, 4.29983548e+00],
       [2.62740449e+00, 7.37257477e-18, 3.00771562e+02, 1.28851737e+00,
        7.37257477e-18, 5.70967031e+00],
       [1.51402470e+00, 2.29346401e-04, 1.50264279e+02, 9.21884935e-01,
        2.29320106e-04, 5.01902850e+00],
       [1.17430057e+00, 3.99961330e-01, 2.24223771e-01, 7.76707036e-01,
        3.36444615e-01, 2.02306987e-01]])

In [51]:
# add labels to customers dataframe and add_points array
add_points = np.append(add_points, [[0], [1], [2], [3]], axis=1)
customers["clusters"] = updated_kmeans_model.labels_

In [52]:
# create centers dataframe from add_points
centers_df = pd.DataFrame(data=add_points, columns=["products_ordered",
                                                    "average_return_rate",
                                                    "total_spending",
                                                    "log_products_ordered",
                                                    "log_average_return_rate",
                                                    "log_total_spending",
                                                    "clusters"])
centers_df.head()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending,clusters
0,1.000373,6.210410e-05,72.687670,0.693333,6.210218e-05,4.299835,0.0
1,2.627404,7.372575e-18,300.771562,1.288517,7.372575e-18,5.709670,1.0
2,1.514025,2.293464e-04,150.264279,0.921885,2.293201e-04,5.019028,2.0
3,1.174301,3.999613e-01,0.224224,0.776707,3.364446e-01,0.202307,3.0


In [53]:
# align cluster centers of centers_df and customers
centers_df["clusters"] = centers_df["clusters"].astype("int")

In [54]:
centers_df.head()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending,clusters
0,1.000373,6.210410e-05,72.687670,0.693333,6.210218e-05,4.299835,0
1,2.627404,7.372575e-18,300.771562,1.288517,7.372575e-18,5.709670,1
2,1.514025,2.293464e-04,150.264279,0.921885,2.293201e-04,5.019028,2
3,1.174301,3.999613e-01,0.224224,0.776707,3.364446e-01,0.202307,3


In [55]:
customers.head()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending,clusters
0,1,0.0,79.2,0.693147,0.0,4.384524,0
1,1,0.0,79.2,0.693147,0.0,4.384524,0
2,2,0.0,194.2,1.098612,0.0,5.274025,2
3,3,0.0,386.8,1.386294,0.0,5.960490,1
4,1,0.0,70.0,0.693147,0.0,4.262680,0


In [56]:
# differentiate between data points and cluster centers
customers["is_center"] = 0
centers_df["is_center"] = 1

# add dataframes together
customers = customers.append(centers_df, ignore_index=True)

In [57]:
customers.tail()

,products_ordered,average_return_rate,total_spending,log_products_ordered,log_average_return_rate,log_total_spending,clusters,is_center
10050,1.000000,0.000000e+00,64.000000,0.693147,0.000000e+00,4.174387,0,0
10051,1.000373,6.210410e-05,72.687670,0.693333,6.210218e-05,4.299835,0,1
10052,2.627404,7.372575e-18,300.771562,1.288517,7.372575e-18,5.709670,1,1
10053,1.514025,2.293464e-04,150.264279,0.921885,2.293201e-04,5.019028,2,1
10054,1.174301,3.999613e-01,0.224224,0.776707,3.364446e-01,0.202307,3,1


### Visualize Customer Segmentation

In [68]:
# add clusters to the dataframe
customers["cluster_name"] = customers["clusters"].astype(str)

In [69]:
# visualize log_transformation customer segments with a 3D plot
fig = px.scatter_3d(customers,
                    x="log_products_ordered",
                    y="log_average_return_rate",
                    z="log_total_spending",
                    color='cluster_name',
                    hover_data=["products_ordered",
                                "average_return_rate",
                                "total_spending"],
                    category_orders = {"cluster_name": 
                                       ["0", "1", "2", "3"]},
                    symbol = "is_center"
                    )

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

## Check for Cluster Magnitude

In [60]:
# values for log_transformation
cardinality_df = pd.DataFrame(
    customers.cluster_name.value_counts().reset_index())

cardinality_df.rename(columns={"index": "Customer Groups",
                               "cluster_name": "Customer Group Magnitude"},
                      inplace=True)

In [61]:
cardinality_df

,Customer Groups,Customer Group Magnitude
0,0,6530
1,2,2510
2,1,996
3,3,19


In [62]:
fig = px.bar(cardinality_df, x="Customer Groups", 
             y="Customer Group Magnitude",
             color = "Customer Groups",
             category_orders = {"Customer Groups": ["0", "1", "2", "3"]})

fig.update_layout(xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1),
                 yaxis = dict(
        tickmode = 'linear',
        tick0 = 1000,
        dtick = 1000))

fig.show()